# Analisis Tagihan Kesehatan pada Asuransi X

by Tria Sofa Purnama (#triasofa)

## Data Description

Analisis akan dilakukan pada data tagihan kesehatan yang diterima oleh setiap pengguna. Data ini berisi data personal pengguna seperti umur, gender, tempat tinggal pengguna, banyak anak tertanggung asuransi, nilai bmi, keadaan merokok atau tidaknya pengguna. Secara rinci, deskripsi setiap feature dari dataset adalah sebagai berikut:
1. age: Age of primary beneficiary
2. sex: Insurance contractor gender, female, male
3. bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg/m2) using the ratio of height to weight, ideally 18.5 to 24.9
4. children: Number of children covered by health insurance / Number of dependents
5. smoker: Smoking
6. region: The beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
7. charges: Individual medical costs billed by health insurance

In [33]:
#Import Tools yang akan digunakan
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics as sta
from scipy import stats

In [3]:
#Upload Data dalam bentuk csv
df = pd.read_csv(r'/Users/triasofa/Documents/Pacmann/Insurance.csv')
print(df)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


## Langkah 1 - Analisa Descriptive Statistic

1. Rata-rata usia
2. Rata-rata nilai BMI dari pengguna yang merokok?
4. Berapa rata rata nilai BMI dari yang merokok?
6. Apakah rata rata umur perempuan dan laki-laki yang merokok sama?
9. BMI mana yang lebih tinggi, seseorang laki-laki atau perempuan?
10. BMI mana yang lebih tinggi, seseorang perokok atau non perokok?

In [7]:
print('Rata-rata usia: ', df['age'].mean())

Rata-rata usia:  39.20702541106129


Berdasarkan data tagihan kesehatan, pada feature numerik rata-rata usia pengguna adalah 39 tahun.

In [10]:
print('Rata-rata BMI yang merokok:',df[df['smoker']=='yes']['bmi'].mean())

Rata-rata BMI yang merokok: 30.708448905109503


In [13]:
print('Rata-rata usia: ', df[df['smoker']=='yes']['age'].mean())
print('Usia terkecil: ', df[df['smoker']=='yes']['age'].min())
print('Usia terbesar: ',df[df['smoker']=='yes']['age'].max())

Rata-rata usia:  38.51459854014598
Usia terkecil:  18
Usia terbesar:  64


Berdasarkan output di atas, didapatkan bahwa pada kondisi merokok rata-rata usia female dan male sama. Rata-rata usia pada female yang merokok maupun male yang merokok adalah 38 tahun.

In [15]:
df.groupby('smoker', axis=0).mean()[['bmi','charges']]

,bmi,charges
smoker,,
no,30.651795,8434.268298
yes,30.708449,32050.231832


BMI pada pengguna yang merokok dan yang tidak merokok cenderung sama. Namun, tagihan kesehatan pada pengguna yang merokok lebih besar daripada tagihan yang tidak merokok.

In [16]:
df.groupby('smoker', axis=0).max()[['bmi','charges']]

,bmi,charges
smoker,,
no,53.13,36910.60803
yes,52.58,63770.42801


Berdasarkan nilai maksimum BMI tertinggi pada pengguna yang tidak merokok. Namun, tagihan kesehatan lebih tinggi pada pengguna yang merokok.

## Langkah 2 - Analisa Variabel Kategorik (PMF)

1. Gender mana yang memiliki tagihan paling tinggi?
2. Distribusi peluang tagihan di tiap-tiap region
3. Apakah setiap region memiliki proporsi data banyak orang yang sama?
4. Mana yang lebih tinggi proporsi perokok atau non perokok?
5. Berapa peluang seseorang tersebut adalah perempuan diketahui dia adalah perokok?

In [18]:
df.groupby('sex', axis=0)[['charges']].aggregate(['mean', 'max'])

charges             
                mean          max
sex                              
female  12569.578844  63770.42801
male    13956.751178  62592.87309

Secara umum, rata-rata tagihan paling tinggi ada pada jenis kelamin male, sedangkan berdasarkan nilai maksimum jenis kelamin female memiliki tagihan paling tinggi.

In [19]:
df1 = df.groupby('region', axis=0)['age'].aggregate(['count'])
df1['peluang'] = df1.transform(lambda x: x/x.sum())
df1

,count,peluang
region,,
northeast,324,0.242152
northwest,325,0.242900
southeast,364,0.272048
southwest,325,0.242900


Distribusi peluang tagihan di tiap-tiap region sebagaimana pada tabel. Proporsi tertinggi berada pada region Southeast dengan proporsi/peluang 0,272.

In [44]:
df1 = df.groupby('smoker', axis=0)['age'].aggregate(['count'])
df1['proporsi'] = df1.transform(lambda x: x/x.sum())
df1

,count,proporsi
smoker,,
no,1064,0.795217
yes,274,0.204783


## Langkah 3 - Analisa Variabel Kontinu

#### Mana yang lebih mungkin terjadi
Seseorang dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k, atau
Seseorang dengan BMI di bawah 25 mendapatkan tagihan kesehatan di atas 16.7k

In [49]:
# Tagihan di atas 16.7K berdasarkan BMI
df1 = df[df['charges']>16700]
df1['bmi_group'] = df1['bmi'].transform(lambda x : 'BMI>25' if x > 25 else 'BMI<=25')
df1 = df1.groupby('bmi_group', axis=0)['age'].aggregate(['count'])
df1['peluang'] = df1.transform(lambda x: x/x.sum())
df1

/var/folders/5k/vl7nr6rj76z74mwnqnqkfnwh0000gn/T/ipykernel_1695/2059959275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['bmi_group'] = df1['bmi'].transform(lambda x : 'BMI>25' if x > 25 else 'BMI<=25')


,count,peluang
bmi_group,,
BMI<=25,51,0.152695
BMI>25,283,0.847305


Seseorang dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k memiliki peluang lebih tinggi dibandingkan eseorang dengan BMI di bawah 25 mendapatkan tagihan kesehatan di atas 16.7k.

#### Mana yang lebih mungkin terjadi

Seseorang pengguna yang merokok dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k, atau
Seseorang pengguna yang tidak merokok dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k

In [52]:
# Perokok dan non perokok dengan BMI di atas 25
df1 = df[(df['bmi']>25) & (df['charges']>16700)]
df1['status'] = df1['smoker'].transform(lambda x : 'Perokok' if x == 'yes' else 'Non Perokok')
df1 = df1.groupby('status', axis=0)['age'].aggregate(['count'])
df1['peluang'] = df1.transform(lambda x: x/x.sum())
df1

/var/folders/5k/vl7nr6rj76z74mwnqnqkfnwh0000gn/T/ipykernel_1695/2069963834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['status'] = df1['smoker'].transform(lambda x : 'Perokok' if x == 'yes' else 'Non Perokok')


,count,peluang
status,,
Non Perokok,68,0.240283
Perokok,215,0.759717


Seseorang pengguna yang merokok dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k memiliki peluang yang lebih besar daripada Seseorang pengguna yang tidak merokok dengan BMI di atas 25 mendapatkan tagihan kesehatan di atas 16.7k.

In [24]:
# Perokok dan non perokok dengan BMI di atas 25
df1 = df[(df['bmi']>25) & (df['charges']>16700)]
df1['status'] = df1['smoker'].transform(lambda x : 'Perokok' if x == 'yes' else 'Non Perokok')
df1 = df1.groupby('status', axis=0)['age'].aggregate(['count'])
df1['peluang'] = df1.transform(lambda x: x/x.sum())
df1

/var/folders/5k/vl7nr6rj76z74mwnqnqkfnwh0000gn/T/ipykernel_1695/2069963834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['status'] = df1['smoker'].transform(lambda x : 'Perokok' if x == 'yes' else 'Non Perokok')


,count,peluang
status,,
Non Perokok,68,0.240283
Perokok,215,0.759717


### Langkah 4 - Analisis Korelasi

In [26]:
df[['age','bmi','children','charges']].corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


Pada tabel di atas menunjukkan korelasi antara feature charges dengan feature age, bmi, dan children. Dari tabel di atas dapat diambil kesimpulan sebagai berikut:
1. age memiliki korelasi lemah dan positif terhadap charges
2. bmi memiliki korelasi lemah dan positif terhadap charges
3. children memiliki korelasi lemah dan positif terhadap charges

### Langkah 5 - Pengujian Hipotesis

In [53]:
alpha = 0.05
H0 = "Tagihan kesehatan pengguna yang merokok = tagihan kesehatan pengguna yang tidak merokok"
H1 = "Tagihan kesehatan pengguna yang merokok lebih tinggi dari pengguna yang tidak merokok"

stat, p = stats.ttest_ind(df[df['smoker']=='yes']['charges'], df[df['smoker']=='no']['charges'])
print('Statistik: ', stat)
print('p-value: ', p)

Statistik:  46.66492117272371
p-value:  8.271435842179102e-283


In [54]:
# Pengambilan keputusan
if p>alpha:
    print("HO diterima, yaitu: ", H0)
else:
    print("H0 ditolak H1 diterima, yaitu: ", H1)

H0 ditolak H1 diterima, yaitu:  Tagihan kesehatan pengguna yang merokok lebih tinggi dari pengguna yang tidak merokok


In [41]:
H0 = "Tagihan kesehatan dengan BMI di atas 25 = tagihan kesehatan dengan BMI di bawah 25"
H1 = "Tagihan kesehatan dengan BMI di atas 25 lebih tinggi daripada tagihan kesehatan dengan BMI di bawah 25"

stat, p = stats.ttest_ind(df[df['bmi']>25]['charges'], df[df['bmi']<=25]['charges'])
print('Statistik: ', stat)
print('p-value: ', p)

Statistik:  4.319946909530397
p-value:  1.6761321969419405e-05


In [42]:
# Pengambilan keputusan
if p>alpha:
    print("HO diterima, yaitu: ", H0)
else:
    print("H0 ditolak H1 diterima, yaitu: ", H1)

H0 ditolak H1 diterima, yaitu:  Tagihan kesehatan dengan BMI di atas 25 lebih tinggi daripada tagihan kesehatan dengan BMI di bawah 25


In [43]:
H0 = "Tagihan kesehatan laki-laki = tagihan kesehatan perempuan"
H1 = "Tagihan kesehatan laki-laki lebih besar dari tagihan kesehatan perempuan"

stat, p = stats.ttest_ind(df[df['sex']=='male']['charges'], df[df['sex']=='female']['charges'])
print('Statistik: ', stat)
print('p-value: ', p)

Statistik:  2.097546590051688
p-value:  0.03613272100592976


In [40]:
if p>alpha:
    print("HO diterima, yaitu: ", H0)
else:
    print("H0 ditolak H1 diterima, yaitu: ", H1)

H0 ditolak H1 diterima, yaitu:  Tagihan kesehatan laki-laki lebih besar dari tagihan kesehatan perempuan
